In [23]:
#@title Define Battery and Solar Parameters

energy_capacity = 720 #@param {type:"number"}
#@markdown > The maximum volume of energy that can be stored in the battery system, measured in MWh. Note that this is the quantity of energy stored in the system after charging efficiency losses and before discharging efficiency losses.

charge_power_limit = 360 #@param {type:"number"}
#@markdown > The maximum power rate at which the battery can charge, measured in MW.

discharge_power_limit = 360 #@param {type:"number"}
#@markdown >  The maximum power rate at which the battery can discharge, measured in MW.

charge_efficiency = 0.95 #@param {type:"number"}
#@markdown > The efficiency at which energy can enter the battery. For example, charging at 1 MW for 1 hour with a 95% charge efficiency will result in 0.95 MWh of energy stored in the battery.

discharge_efficiency = 0.95 #@param {type:"number"}
#@markdown > The efficiency at which energy can leave the battery. For example, discharging at 1 MW for 1 hour with a 95% discharge efficiency will result in 0.95 MWh of energy to the grid.

SOC_max_percentage = 100 #@param {type:"number"}
#@markdown > The maximum allowable percentage of energy that can be stored in the battery, relative to the Energy Capacity.

SOC_min_percentage = 10 #@param {type:"number"}
#@markdown > The minimum allowable percentage of energy that can be stored in the battery, relative to the Energy Capacity.

SOC_max = (SOC_max_percentage / 100) * energy_capacity
SOC_min = (SOC_min_percentage / 100) * energy_capacity

daily_cycle_limit = 1 #@param {type:"number"}
#@markdown > The maximum number of cycles allowed in a day. This constraint can be imposed for battery health reasons.

annual_cycle_limit = 400 #@param {type:"number"}
#@markdown > The maximum number of cycles allowed in a year. This constraint can be imposed for battery warranty or long-term degradation limiting reasons.

solar_dc_capacity = 1120 #@param {type:"number"}
#@markdown > The maximum power generation capacity of the solar system, measured in MW.

maximum_export_capacity = 800 #@param {type:"number"}
#@markdown > The maximum power that can be exported to the grid, measured in MW.

SOC_initial = SOC_min
#@markdown > The initial State of Charge (SOC) of the battery, set to the minimum SOC. This is a percentage of the overall energy capacity converted to MWh.


In [24]:
#@title Load data, scale solar, calculate dynamic export capacity

import pandas as pd
import requests
from io import StringIO

# Assuming solar_dc_capacity is defined and has a value

# Load and prepare the data
file_id = "1fCNAptFf_yTjLAlTMDXTFErR1ndV7dAx"
url = f"https://drive.google.com/uc?export=download&id={file_id}"
response = requests.get(url)
assert response.status_code == 200, 'Wrong status code'
data = StringIO(response.text)
df = pd.read_csv(data)
df['Date, time'] = pd.to_datetime(df['Date, time'])
df = df.rename(columns={'LMP': 'lmp'})

# Scale the solar data
original_capacity = 1040  # Original capacity in MWs
df['System Export - KW (800)'] = (df['System Export - KW (800)'] / original_capacity) * solar_dc_capacity

# Ensure that the scaled values do not exceed 800,000 kW
df['System Export - KW (800)'] = df['System Export - KW (800)'].clip(upper=800000)

# Select date range
start_date = '2027-07-01' #@param {type:"date"}
end_date = '2029-07-01' #@param {type:"date"}

# Filter the DataFrame based on the provided dates
df_filtered = df[(df['Date, time'] >= start_date) & (df['Date, time'] <= end_date)]


# Function to calculate maximum export capacity
def calculate_max_export_capacity(df, max_export_capacity_mw, solar_export_column):
    # Convert solar production from KW to MW and change the sign
    df['Solar_Production_MW'] = -df[solar_export_column] / 1000

    # Calculate the remaining export capacity
    df['Max_Export_Capacity_MW'] = max_export_capacity_mw + df['Solar_Production_MW']

    return df

# Calculate the maximum export capacity for the filtered DataFrame
max_export_capacity_mw = 800  # 800 MW
df_filtered = calculate_max_export_capacity(df_filtered, max_export_capacity_mw, 'System Export - KW (800)')

# Print the first few lines of the transformed data to verify
print(df_filtered.head())




               Date, time    lmp  System Export - KW (800)  \
87648 2027-07-01 00:00:00  61.72              -1177.184615   
87649 2027-07-01 00:30:00  61.34              -1177.184615   
87650 2027-07-01 01:00:00  50.55              -1177.184615   
87651 2027-07-01 01:30:00  50.72              -1177.184615   
87652 2027-07-01 02:00:00  54.23              -1177.184615   

       Solar_Production_MW  Max_Export_Capacity_MW  
87648             1.177185              801.177185  
87649             1.177185              801.177185  
87650             1.177185              801.177185  
87651             1.177185              801.177185  
87652             1.177185              801.177185  


<ipython-input-24-81ce999ddedd>:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-24-81ce999ddedd>:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
#@title Standalone Dispatch

!pip install pulp
from pulp import LpVariable, LpProblem, LpMaximize, lpSum, value, LpStatus

# Recalculate da_prices with the updated df_filtered
da_prices = df_filtered['lmp'].tolist()

# Now create the DataFrame for battery dispatch using the updated df_filtered
battery_dispatch_df = pd.DataFrame({
    'interval_start_local': df_filtered['Date, time'],  # Use the date from the updated filtered DataFrame
    'lmp': da_prices
})

# Price Forecast for num_intervals intervals
num_intervals = len(da_prices)
print(f"Number of intervals: {num_intervals}")

num_days = num_intervals / 24  # Adjusted for hourly data
print(f"Number of days: {num_days}")

total_cycle_limit = (num_days / 365) * annual_cycle_limit
print(f"Total cycle limit: {total_cycle_limit}")

# Variables
charge_vars = LpVariable.dicts("Charging", range(num_intervals), lowBound=0, upBound=charge_power_limit)
discharge_vars = LpVariable.dicts("Discharging", range(num_intervals), lowBound=0, upBound=discharge_power_limit)
SOC_vars = LpVariable.dicts("SOC", range(num_intervals+1), lowBound=SOC_min, upBound=SOC_max)  # Including initial SOC

# Problem
prob = LpProblem("Battery Scheduling", LpMaximize)

# Objective function
prob += lpSum([da_prices[t]*discharge_efficiency*discharge_vars[t] - da_prices[t]*charge_vars[t]/charge_efficiency for t in range(num_intervals)])

# Constraints
# Initial SOC constraint
prob += SOC_vars[0] == SOC_initial

# SOC update constraints
for t in range(num_intervals):
    prob += SOC_vars[t+1] == SOC_vars[t] + charge_efficiency * charge_vars[t] - discharge_efficiency * discharge_vars[t]


# Charge/Discharge constraints based on SOC
for t in range(num_intervals):
    prob += SOC_vars[t] + charge_efficiency*charge_vars[t] <= SOC_max  # Cannot charge if SOC_max is reached
    prob += SOC_vars[t] - discharge_vars[t]*discharge_efficiency >= SOC_min  # Cannot discharge below SOC_min

# Simultaneous charge and discharge constraint
for t in range(num_intervals):
    prob += charge_vars[t] + discharge_vars[t] <= max(charge_power_limit, discharge_power_limit)

# Daily cycle limit constraint
for day_start in range(0, num_intervals, 24):
    day_end = min(day_start + 24, num_intervals)
    prob += lpSum([charge_vars[t] for t in range(day_start, day_end)]) * charge_efficiency / energy_capacity <= daily_cycle_limit

# Annual cycle limit constraint
prob += lpSum([charge_vars[t] for t in range(num_intervals)]) * charge_efficiency / energy_capacity <= total_cycle_limit

# Solve the problem
prob.solve()

# Check the status of the solution
print("Status:", LpStatus[prob.status])

# Assuming the problem is solved successfully, the rest of the code can proceed
# However, you should include checks to ensure the problem was solved correctly before proceeding

# Create the battery dispatch DataFrame
battery_dispatch_df = pd.DataFrame({
    'interval_start_local': df_filtered['Date, time'],  # Correct column name
    'lmp': da_prices
})

# Create series from the optimization results for the unconstrained scenario
discharge_vars_series_unconstrained = pd.Series([value(discharge_vars[t]) if discharge_vars[t].varValue is not None else 0 for t in range(num_intervals)], name='discharge_vars')
charge_vars_series_unconstrained = pd.Series([value(charge_vars[t]) if charge_vars[t].varValue is not None else 0 for t in range(num_intervals)], name='charge_vars')
soc_vars_series_unconstrained = pd.Series([value(SOC_vars[t]) if SOC_vars[t].varValue is not None else 0 for t in range(num_intervals+1)], name='soc_vars')[:-1]

battery_dispatch_df_unconstrained = pd.DataFrame({
    'discharge_vars': discharge_vars_series_unconstrained.values.round(1),
    'charge_vars': charge_vars_series_unconstrained.values.round(1),
    'SOC_vars': soc_vars_series_unconstrained.values.round(1),
    'lmp': da_prices
}).set_index(df_filtered['Date, time'])  # Use the date from the filtered DataFrame for setting index

# Set the index to 'Date, time' for battery_dispatch_df_unconstrained
battery_dispatch_df_unconstrained.set_index(df_filtered['Date, time'], inplace=True)

# Display the first few rows of the dispatch schedule
print(battery_dispatch_df.head(24))

# Check the problem status and results
print("Status:", LpStatus[prob.status])



Number of intervals: 35089
Number of days: 1462.0416666666667
Total cycle limit: 1602.2374429223746


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning:

Spaces are not permitted in the name. Converted to '_'



Status: Optimal
      interval_start_local    lmp
87648  2027-07-01 00:00:00  61.72
87649  2027-07-01 00:30:00  61.34
87650  2027-07-01 01:00:00  50.55
87651  2027-07-01 01:30:00  50.72
87652  2027-07-01 02:00:00  54.23
87653  2027-07-01 02:30:00  54.53
87654  2027-07-01 03:00:00  52.22
87655  2027-07-01 03:30:00  53.80
87656  2027-07-01 04:00:00  61.80
87657  2027-07-01 04:30:00  62.03
87658  2027-07-01 05:00:00  63.34
87659  2027-07-01 05:30:00  63.02
87660  2027-07-01 06:00:00  61.32
87661  2027-07-01 06:30:00  65.70
87662  2027-07-01 07:00:00  65.28
87663  2027-07-01 07:30:00  66.05
87664  2027-07-01 08:00:00  67.03
87665  2027-07-01 08:30:00  66.88
87666  2027-07-01 09:00:00  64.96
87667  2027-07-01 09:30:00  65.36
87668  2027-07-01 10:00:00  57.79
87669  2027-07-01 10:30:00  56.38
87670  2027-07-01 11:00:00  54.18
87671  2027-07-01 11:30:00  53.68
Status: Optimal


In [26]:
#@title Solar Constrained Dispatch

!pip install pulp
from pulp import LpVariable, LpProblem, LpMaximize, lpSum, value, LpStatus

# Assuming da_prices has been defined in the data preparation code

# Price Forecast for num_intervals intervals
num_intervals = len(da_prices)
print(f"Number of intervals: {num_intervals}")

num_days = num_intervals / 24  # Adjusted for hourly data
print(f"Number of days: {num_days}")

total_cycle_limit = (num_days / 365) * annual_cycle_limit
print(f"Total cycle limit: {total_cycle_limit}")

# Variables
charge_vars = LpVariable.dicts("Charging", range(num_intervals), lowBound=0, upBound=charge_power_limit)
discharge_vars = LpVariable.dicts("Discharging", range(num_intervals), lowBound=0, upBound=discharge_power_limit)
SOC_vars = LpVariable.dicts("SOC", range(num_intervals+1), lowBound=SOC_min, upBound=SOC_max)  # Including initial SOC

# Problem
prob = LpProblem("Battery Scheduling", LpMaximize)

# Objective function
prob += lpSum([da_prices[t]*discharge_efficiency*discharge_vars[t] - da_prices[t]*charge_vars[t]/charge_efficiency for t in range(num_intervals)])

# Constraints
# Initial SOC constraint
prob += SOC_vars[0] == SOC_initial

# SOC update constraints
for t in range(num_intervals):
    prob += SOC_vars[t+1] == SOC_vars[t] + charge_efficiency * charge_vars[t] - discharge_efficiency * discharge_vars[t]


# Charge/Discharge constraints based on SOC
for t in range(num_intervals):
    prob += SOC_vars[t] + charge_efficiency*charge_vars[t] <= SOC_max  # Cannot charge if SOC_max is reached
    prob += SOC_vars[t] - discharge_vars[t]*discharge_efficiency >= SOC_min  # Cannot discharge below SOC_min

# Simultaneous charge and discharge constraint
for t in range(num_intervals):
    prob += charge_vars[t] + discharge_vars[t] <= max(charge_power_limit, discharge_power_limit)

# Daily cycle limit constraint
for day_start in range(0, num_intervals, 24):
    day_end = min(day_start + 24, num_intervals)
    prob += lpSum([charge_vars[t] for t in range(day_start, day_end)]) * charge_efficiency / energy_capacity <= daily_cycle_limit

# Annual cycle limit constraint
prob += lpSum([charge_vars[t] for t in range(num_intervals)]) * charge_efficiency / energy_capacity <= total_cycle_limit

# Modify the Constraints to Include max_export_capacity
for t in range(num_intervals):
    # Access the max export capacity for the corresponding time interval
    max_export_capacity_at_t = df_filtered['Max_Export_Capacity_MW'].iloc[t]
    prob += discharge_vars[t] <= max_export_capacity_at_t


# Solve the problem
prob.solve()

# Check the status of the solution
print("Status:", LpStatus[prob.status])

# Print a snippet of the filtered DataFrame to verify column data
print("Sample data from df_filtered:")
print(df_filtered[['Date, time', 'lmp', 'System Export - KW (800)', 'Max_Export_Capacity_MW']].head())

# Check the first few prices used in the optimization model
print("\nFirst few LMP values used for optimization:")
print(da_prices[:5])

# Assuming the problem is solved successfully, the rest of the code can proceed
# However, you should include checks to ensure the problem was solved correctly before proceeding

# Create the battery dispatch DataFrame
battery_dispatch_df = pd.DataFrame({
    'interval_start_local':df_filtered['Date, time'],
    'lmp': da_prices
})

# Create series from the optimization results for the constrained scenario
discharge_vars_series_constrained = pd.Series([value(discharge_vars[t]) if discharge_vars[t].varValue is not None else 0 for t in range(num_intervals)], name='discharge_vars')
charge_vars_series_constrained = pd.Series([value(charge_vars[t]) if charge_vars[t].varValue is not None else 0 for t in range(num_intervals)], name='charge_vars')
soc_vars_series_constrained = pd.Series([value(SOC_vars[t]) if SOC_vars[t].varValue is not None else 0 for t in range(num_intervals+1)], name='soc_vars')[:-1]

# Create a DataFrame for the constrained scenario
battery_dispatch_df_constrained = pd.DataFrame({
    'discharge_vars': discharge_vars_series_constrained.values.round(1),
    'charge_vars': charge_vars_series_constrained.values.round(1),
    'SOC_vars': soc_vars_series_constrained.values.round(1),
    'lmp': da_prices
})

# Set the index to 'Date, time' from the filtered DataFrame
battery_dispatch_df_constrained.set_index(df_filtered['Date, time'], inplace=True)

# Display the first few rows of the dispatch schedule
print(battery_dispatch_df.head(24))

# Check the problem status and results
print("Status:", LpStatus[prob.status])

# Check the `max_export_capacity_at_t` for any anomalies
print("Sample data for Max Export Capacity MW:")
print(df_filtered[['Date, time', 'Max_Export_Capacity_MW']].head(10))

# Check if all `max_export_capacity_at_t` are positive
if all(df_filtered['Max_Export_Capacity_MW'] > 0):
    print("All max export capacity values are positive.")
else:
    print("There are non-positive values in max export capacity which could make the problem infeasible.")

# Check the range of `max_export_capacity_at_t`
print("Range of Max Export Capacity MW:")
print(df_filtered['Max_Export_Capacity_MW'].agg(['min', 'max']))


Number of intervals: 35089
Number of days: 1462.0416666666667
Total cycle limit: 1602.2374429223746


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning:

Spaces are not permitted in the name. Converted to '_'



Status: Optimal
Sample data from df_filtered:
               Date, time    lmp  System Export - KW (800)  \
87648 2027-07-01 00:00:00  61.72              -1177.184615   
87649 2027-07-01 00:30:00  61.34              -1177.184615   
87650 2027-07-01 01:00:00  50.55              -1177.184615   
87651 2027-07-01 01:30:00  50.72              -1177.184615   
87652 2027-07-01 02:00:00  54.23              -1177.184615   

       Max_Export_Capacity_MW  
87648              801.177185  
87649              801.177185  
87650              801.177185  
87651              801.177185  
87652              801.177185  

First few LMP values used for optimization:
[61.72, 61.34, 50.55, 50.72, 54.23]
      interval_start_local    lmp
87648  2027-07-01 00:00:00  61.72
87649  2027-07-01 00:30:00  61.34
87650  2027-07-01 01:00:00  50.55
87651  2027-07-01 01:30:00  50.72
87652  2027-07-01 02:00:00  54.23
87653  2027-07-01 02:30:00  54.53
87654  2027-07-01 03:00:00  52.22
87655  2027-07-01 03:30:00  53.80
87

In [27]:
#@title Compare Results

import plotly.graph_objects as go

# Define a function to calculate the monthly metrics
def calculate_monthly_metrics(df):
    df['hourly_discharging_revenue'] = df['discharge_vars'] * df['lmp'] * discharge_efficiency
    df['hourly_charging_costs'] = df['charge_vars'] * df['lmp'] / charge_efficiency
    df['hourly_net_revenue'] = df['hourly_discharging_revenue'] - df['hourly_charging_costs']

    monthly_metrics = df[['hourly_discharging_revenue', 'hourly_charging_costs', 'hourly_net_revenue']].resample('MS').sum()
    monthly_metrics['Net Revenue (£)'] = monthly_metrics['hourly_net_revenue']

    return monthly_metrics[['Net Revenue (£)']]

# Calculate monthly metrics for both scenarios
monthly_metrics_unconstrained = calculate_monthly_metrics(battery_dispatch_df_unconstrained)
monthly_metrics_constrained = calculate_monthly_metrics(battery_dispatch_df_constrained)

# Combine metrics into a single DataFrame
comparison_df = pd.concat([monthly_metrics_unconstrained, monthly_metrics_constrained], axis=1)
comparison_df.columns = ['Net Revenue (£) Unconstrained', 'Net Revenue (£) Constrained']

# Calculate the percentage difference and add it to the comparison DataFrame
comparison_df['% Difference'] = ((comparison_df['Net Revenue (£) Unconstrained'] - comparison_df['Net Revenue (£) Constrained']) / comparison_df['Net Revenue (£) Unconstrained']) * 100

# Update the table generation code to include the new column
table = go.Figure(data=[go.Table(
    header=dict(values=comparison_df.columns.insert(0, 'Month'),
                fill_color='black',
                font=dict(color='white'),
                align='left'),
    cells=dict(values=[comparison_df.index.strftime('%Y-%m')] + [comparison_df[col].apply(lambda x: f"£{x:,.0f}" if col.startswith('Net Revenue') else f"{x:.2f}%" ) for col in comparison_df.columns],
               fill_color='darkslategray',
               font=dict(color='white'),
               align='left'))
])

# Render the table
table.show()


# BAR CHART
total_revenue_unconstrained = monthly_metrics_unconstrained['Net Revenue (£)'].sum()
total_revenue_constrained = monthly_metrics_constrained['Net Revenue (£)'].sum()

# Calculate overall percentage difference
overall_percentage_difference = ((total_revenue_unconstrained - total_revenue_constrained) / total_revenue_unconstrained) * 100

# Create the bar chart
bar_chart = go.Figure(data=[
    go.Bar(name='Unconstrained', x=['Total Revenue'], y=[total_revenue_unconstrained]),
    go.Bar(name='Constrained', x=['Total Revenue'], y=[total_revenue_constrained])
])

# Update the layout for the bar chart
bar_chart.update_layout(
    title_text=f"Overall Revenue Comparison - {overall_percentage_difference:.2f}% Difference",
    template="plotly_dark",
    yaxis_title="Total Revenue (£)"
)

# Render the bar chart
bar_chart.show()


